In [54]:
import requests
import random
import string
import json
import time


def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"调用chatgpt模型耗时{end_time - start_time:.6f} s.")
        return result

    return wrapper


@timer_decorator
def send_chat_message(input_text, avoidRepeatable):
    url = "https://api.nextera.chat/webflux/api/chat"

    headers = {
        'Gpt-Auth-Token': '1c855e967175a518',
        'Content-Type': 'application/json',
        'Accept': 'text/event-stream'
    }

    data = {
        "sessionType": "chat",
        "settingCode": "55c161ef53ff7dfe3fb425a45feebee3",
        "sessionIdentifier": "61dce617c5d3422285e0badfc9964d71",
        "stream": "true",
        "content": input_text,
        "callbackUrl": "callback",
        "avoidRepeatable": avoidRepeatable
    }

    response = requests.post(url, headers=headers, json=data)
    return response


def split_chat_json(data):
    json_blocks = data.strip().split("\ndata:")
    # 取最后一个 JSON 数据块
    last_json_data = json_blocks[-1]

    last_json_object = json.loads(last_json_data)
    output = last_json_object['result']['choices'][0]['message']['content']
    return output

def topic_extraction_model_results(input_text):
    # 调用函数并传入 avoidRepeatable 参数
    common_input_dic_text = """
    假设你现在是一个主题提取模型，当我输入的是一句话时，你只输出这段文字对应的意图和实体。
    现在有两个主题，一个是问某种事物是什么概念，一个是求销量。

    问销量的例子如下，当输入为“我想知道前三个月大众汽车的销量平均值”时的具体输出示例：{
    "主题": "销量计算问题",   
    "统计指标": "平均值",  
     "时间": "前三个月",  
     "车辆": "大众"}。

    问概念的例子如下，当输入为“我想知道进站客户是什么意思“时的具体输出示例：{
    "主题": "概念问题",   
    "名词": "进站客户"}
    
    如果不满足以上主题，输出：{
    "主题": "未知",   
    "回复": "抱歉,我没有听清，麻烦您再说一次"}

     以下是我的提问："""
    input_text = common_input_dic_text + input_text
    avoidRepeatable_value = ''.join(random.choice(string.digits) for _ in range(10))
    try:
        response = send_chat_message(input_text, avoidRepeatable_value)
        output = split_chat_json(response.text)
        return output
    except Exception as e:
        print(e)

In [55]:
if __name__ == '__main__':
    
    input_text_01 = "宝马汽车的前3个月的销量总和是多少？"
    input_text_02 = "库存安全线是什么意思？"
    input_text_03 = "1+1？"

    print(topic_extraction_model_results(input_text_01))
    print(topic_extraction_model_results(input_text_02))
    print(topic_extraction_model_results(input_text_03))

调用chatgpt模型耗时13.395453 s.
{
    "主题": "销量计算问题",
    "统计指标": "总和",
    "时间": "前3个月",
    "车辆": "宝马"
}
调用chatgpt模型耗时10.422753 s.
{
    "主题": "概念问题",
    "名词": "库存安全线"
}
调用chatgpt模型耗时13.596737 s.
{
    "主题": "未知",
    "回复": "抱歉,我没有听清，麻烦您再说一次"
}


In [67]:
import requests
import json

API_KEY = "S883mOp9aOTvjeeyy53Pll9L"
SECRET_KEY = "ZgPpQUgeDvnd0d81kXY06Z3tb5RAqX1X"

input_text ="""假设你现在是一个主题提取模型，当我输入的是一句话时，你只输出这段文字对应的意图和实体。
    现在有两个主题，一个是问某种事物是什么概念，一个是求销量。

    问销量的例子如下，当输入为“我想知道前三个月大众汽车的销量平均值”时的具体输出示例：{
    "主题": "销量计算问题",   
    "统计指标": "平均值",  
     "时间": "前三个月",  
     "车辆": "大众"}。

    问概念的例子如下，当输入为“我想知道进站客户是什么意思“时的具体输出示例：{
    "主题": "概念问题",   
    "名词": "进站客户"}
    
    如果不满足以上主题，输出：{
    "主题": "未知",   
    "回复": "抱歉,我没有听清，麻烦您再说一次"}

     以下是我的提问：今天星期几
     """
def main():
        
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/bloomz_7b1?access_token=" + get_access_token()
    
    payload = json.dumps({
        "messages": [
            {
                "role": "user",
                "content": input_text
            }
        ]
    })
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    
    print(response.text)
    

def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    return str(requests.post(url, params=params).json().get("access_token"))

if __name__ == '__main__':
    main()


{"error_code":17,"error_msg":"Open api daily request limit reached"}
